In [12]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast
from datetime import datetime

In [13]:
df = pd.read_csv(f"output/features_labeled_4llm.csv")

In [14]:
df.head()

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list_inhealthy_lessoften,result_list_inhealthy_moreoften,root_cause,inject_comp_tdiff,nezha_rank_050
0,2022-08-22 03:54,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '23_106', 'score': 0.5400894187779...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",['CpuUsageRate(%)'],1,1
1,2022-08-22 03:55,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",['CpuUsageRate(%)'],2,1
2,2022-08-22 04:03,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...","['Serving product page started', 'GetProduct s...",1,2
3,2022-08-22 04:04,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '18_191', 'score': 1.0, 'deepth': ...","[{'events': '120_198', 'score': 1.0, 'deepth':...","['Serving product page started', 'GetProduct s...",2,2
4,2022-08-22 04:11,frontend-579b9bff58-t2dbm,2022-08-22 04:10:20,cpu_consumed,2022-08-22 03:51,"[{'events': '76_197', 'score': 1.0, 'deepth': ...","[{'events': '104_106', 'score': 0.605582524271...",['CpuUsageRate(%)'],1,4


In [15]:
df['result_list_inhealthy_lessoften'] = df['result_list_inhealthy_lessoften'].apply(lambda x: ast.literal_eval(x))
df['result_list_inhealthy_moreoften'] = df['result_list_inhealthy_moreoften'].apply(lambda x: ast.literal_eval(x))

In [16]:
def format_result_list_inhealthy_moreoften(result_list):
    """
    Format result list for LLM comprehension
    """
    for i in range(len(result_list)):
        entry = result_list[i]
        entry['events_expected'] = entry.pop('events')
    
    return result_list

In [17]:
df['result_list_inhealthy_moreoften'] = df['result_list_inhealthy_moreoften'].apply(format_result_list_inhealthy_moreoften)

In [18]:
def format_result_list_inhealthy_lessoften(result_list):
    """
    Format result list for LLM comprehension
    """
    for i in range(len(result_list)):
        entry = result_list[i]
        if 'events_actual' in entry:
            entry['events_expected'] = entry.pop('events_actual')
        entry['events_actual'] = entry.pop('events')
        entry['score'] = entry['score'] * -1
    
    return result_list

In [19]:
df['result_list_inhealthy_lessoften'] = df['result_list_inhealthy_lessoften'].apply(format_result_list_inhealthy_lessoften)

In [21]:
pd.set_option("display.max_colwidth", None)
df[['result_list_inhealthy_moreoften']].head()

,result_list_inhealthy_moreoften
0,"[{'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '76_197', 'events_expected': '76_4'}, {'score': 0.5638418079096045, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '104_23', 'events_expected': '104_106'}, {'score': 0.52, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '23_20', 'events_expected': '23_100'}, {'score': 0.5140495867768595, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_expected': '120_198'}, {'score': 0.5100250626566416, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': '98_100'}]"
1,"[{'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '76_197', 'events_expected': '76_4'}, {'score': 0.7028169014084507, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': '104_106'}, {'score': 0.6, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': '24_100'}, {'score': 0.5992865636147443, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': '104_100'}, {'score': 0.5738007380073801, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_expected': '120_198'}]"
2,"[{'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_expected': '120_198'}, {'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_actual': '28_208', 'events_expected': '28_119'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_78'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_17'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_80'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_82'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_84'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_86'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '16_41', 'events_expected': '16_15'}, {'score': 0.75, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_expected': '24_100'}]"
3,"[{'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_expected': '120_198'}, {'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_actual': '28_208', 'events_expected': '28_119'}, {'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_actual': '28_208', 'events_expected': '28_120'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_78'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_17'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_80'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_82'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_84'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_191', 'events_expected': '18_86'}, {'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '16_41', 'events_expected': '16_15'}]"
4,"[{'score': 0.6055825242718447, 'deepth': 2, 'pod': 'currencyservice-cf787dd4

In [22]:
df.columns

Index(['abnormal_time', 'inject_pod', 'inject_time', 'inject_type',
       'normal_time', 'result_list_inhealthy_lessoften',
       'result_list_inhealthy_moreoften', 'root_cause', 'inject_comp_tdiff',
       'nezha_rank_050'],
      dtype='object')

In [23]:
df.inject_pod.unique()

array(['frontend-579b9bff58-t2dbm', 'cartservice-579f59597d-wc2lz',
       'checkoutservice-578fcf4766-9csqn',
       'currencyservice-cf787dd48-vpjrd', 'emailservice-55fdc5b988-f6xth',
       'paymentservice-9cdb6588f-554sm',
       'productcatalogservice-668d5f85fb-wckp8',
       'recommendationservice-6cfdd55578-gfj6q',
       'shippingservice-7b598fc7d-lmggd', 'adservice-5f6585d649-fnmft',
       'productcatalogservice-668d5f85fb-jhwx9'], dtype=object)

In [24]:
df.to_csv(f"output/features_labeled_4llm_step2.csv", index=False)